<a href="https://colab.research.google.com/github/otabekds/Classification-projects/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [18]:
import pandas as pd 
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.ensemble import RandomForestClassifier

#Data learning section 

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Data science/real_projects/Identify diabetes/Identify_diabetes_data.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [6]:
df.corr()['Outcome'].sort_values(ascending=False)

Outcome                     1.000000
Glucose                     0.466581
BMI                         0.292695
Age                         0.238356
Pregnancies                 0.221898
DiabetesPedigreeFunction    0.173844
Insulin                     0.130548
SkinThickness               0.074752
BloodPressure               0.065068
Name: Outcome, dtype: float64

In [7]:
df.drop(['SkinThickness','BloodPressure','Insulin','DiabetesPedigreeFunction'], axis=1, inplace=True)

In [8]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [9]:
### In this case, we equalize our data 0 and 1, otherwise our data will be skewed to one side
w = df[df.Outcome == 0]
idx = w.iloc[:232].index
df.drop(idx, inplace=True)

df.Outcome.value_counts()

1    268
0    268
Name: Outcome, dtype: int64

#Preparing data for ML section 

In [10]:
x_set = df.drop("Outcome", axis=1)
y_set = df.Outcome

In [11]:
scaler = MinMaxScaler()
x_set = scaler.fit_transform(x_set)

#ML prediction section 

In [12]:
knn = KNeighborsClassifier(n_neighbors=12)

In [13]:
###Using GridSearchCV, we select the parameter for the best n_neighbors

parameter = {"n_neighbors" : np.arange(1,25)}
knn_gs = GridSearchCV(estimator= knn, param_grid= parameter, cv=5)
knn_gs.fit(x_set, y_set)

knn_gs.best_params_

{'n_neighbors': 12}

In [14]:
predict = cross_val_predict(estimator=knn, X=x_set, y=y_set, cv=5)

#ML evaluation section 

In [15]:
confusion_matrix(y_set, predict)

array([[212,  56],
       [ 62, 206]])

In [16]:
print(classification_report(y_set, predict))

              precision    recall  f1-score   support

           0       0.77      0.79      0.78       268
           1       0.79      0.77      0.78       268

    accuracy                           0.78       536
   macro avg       0.78      0.78      0.78       536
weighted avg       0.78      0.78      0.78       536



#RandomForestClassifier prediction section 


In [31]:
clf = RandomForestClassifier(max_depth=33, random_state=234)

In [32]:
predict = cross_val_predict(estimator=clf, X=x_set, y=y_set, cv=5)

In [33]:
confusion_matrix(y_set, predict)

array([[209,  59],
       [ 64, 204]])

In [34]:
print(classification_report(y_set, predict))

              precision    recall  f1-score   support

           0       0.77      0.78      0.77       268
           1       0.78      0.76      0.77       268

    accuracy                           0.77       536
   macro avg       0.77      0.77      0.77       536
weighted avg       0.77      0.77      0.77       536

